In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

url = 'https://www.olx.ua/obyavlenie/arenda-vip-kvartiry-na-deribasovskoy-IDAMh6k.html#a206a16f05'
response = requests.get(url).text

soup = BeautifulSoup(response, 'lxml')
table = soup.find_all('table', class_='item')
projects = []
for tab in table:
    projects.append({
        'title':tab.find('th').text.strip(),
        'value':tab.find('td').text.strip()
    })

# Let's collect out data
df = []

df.append({
    'comission' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Без комиссии' else '' for i in range (0,len(projects))]),
    'type_of_building' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Тип объекта' else '' for i in range (0,len(projects))]),
    'floor' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Этаж' else '' for i in range (0,len(projects))]),
    'floor_building' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Этажность' else '' for i in range (0,len(projects))]),
    'space' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Общая площадь' else '' for i in range (0,len(projects))]),
    'rooms' : ''.join([projects[i]['value'] if projects[i]['title'] == 'Количество комнат' else '' for i in range (0,len(projects))]),
    'description' : soup.find_all('p', class_='pding10')[0].text.strip(),
    'fotos': [soup.find_all('div', class_='tcenter img-item')[i].find('img').get('src') for i in range(len(soup.find_all('div', class_='tcenter img-item')))]
})

df_main = pd.DataFrame.from_dict(df)
print(df_main)


  comission                                        description floor  \
0            Предлагается к долгосрочной аренде по-настояще...     3   

  floor_building                                              fotos rooms  \
0              5  [https://img01-olxua.akamaized.net/img-olxua/6...     3   

    space type_of_building  
0  230 м²         Квартира  


In [9]:
# Parse fotos
urls = df_main.loc[0,'fotos']

def get_file(url):
    r = requests.get(url, stream = True)
    return r

def get_name(url):
    name = url.split('/')[-1]
    
    folder = 'For fotos'
    #os.path.exists(folder)
    if not os.path.exists(folder):
        os.makedirs(folder)
    path = os.path.abspath(folder)
    return path + '/' + name

def save_image(name, file_object):
    with open(name, 'bw') as f:
        for chunk in file_object.iter_content(8192):
            f.write(chunk)

def main():
    for url in urls:
        save_image(get_name(url), get_file(url))

if __name__=='__main__':
    main()